[View in Colaboratory](https://colab.research.google.com/github/saitota/ProgrammingKnock100/blob/master/Chapter04_Q30_Q39.ipynb)

# :muscle: 前書き
詳細は[第1章](https://qiita.com/saitotak/items/73852ecb7cc4b10b42a5)参照、引き続き言語処理100本ノックの第3章を回答していきます。本章では TBD


# :muscle: 第4章: 形態素解析
> 夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．
> なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．

UTF-8,LF の1万行程度の日本語プレーンテキストでした。形態素解析と統計のライブラリを存分に使うことになりそうな感じです。

```text:neko.txt
一

　吾輩は猫である。
名前はまだ無い。

　どこで生れたかとんと見当がつかぬ。
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
（以下略）
```

## ⚾ 30. 形態素解析結果の読み込み
> 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．


## 下準備として
- ファイルダウンロード
- janomeのインストール

In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt','./neko.txt')


('./neko.txt', <http.client.HTTPMessage at 0x7f50e934e828>)

In [2]:
!pip install janome

    100% |████████████████████████████████| 20.0MB 1.8MB/s 
  Running setup.py bdist_wheel for janome ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/53/60/be/fe884e2d0ebc9fec0988736cf08a2820ab34e3569fc0c5a25a
Successfully built janome


### 解0-1 janomeによる形態素解析と、neko.txt.janome のテキスト作成（おそい）
- MeCab ではなく janome をインストールします、なぜならインストールが簡単だったから。
- `janome.tokenizer` を使って形態素解析します
    - [janome package — janome.tokenizer](http://mocobeta.github.io/janome/api/janome.html#module-janome.tokenizer)
- やたら遅かったので、開始・終了 時間を計測するコードを入れました
- 実測：42sec


In [3]:
from janome.tokenizer import Tokenizer
import janome
import datetime

print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# WriteのたびにOpen
with open ('./neko.txt','r',encoding='utf-8') as file_temp:
    for line in file_temp:
        # 形態素解析して保存
        for token in t.tokenize(line, stream=True):
            with open ('./neko.txt.janome-01','a',encoding='utf-8') as file_janome:
                    file_janome.write(str(token) + '\n')

print('end:' + str(datetime.datetime.now()))

'''
一	名詞,数,*,*,*,*,一,イチ,イチ
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
（以下略）
'''


start:2018-05-15 05:59:23.543077
end:2018-05-15 06:00:01.161180


'\n一\t名詞,数,*,*,*,*,一,イチ,イチ\n吾輩\t名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n猫\t名詞,一般,*,*,*,*,猫,ネコ,ネコ\nで\t助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ\nある\t助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル\n。\t記号,句点,*,*,*,*,。,。,。\n名前\t名詞,一般,*,*,*,*,名前,ナマエ,ナマエ\n（以下略）\n'

### 解0-2 高速化
- メモリ枯渇を鑑み 形態素解析→write を繰り返すようにしてみましたが、やたらと時間がかかったため、一括で書き込むようにして高速化を図ります
- 実測：29sec

In [4]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# Openしてから
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-02','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        for x in t.tokenize(line, stream=True):
            file_janome.write(str(x) + '\n') 
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-15 06:00:01.826460
end:2018-05-15 06:00:27.261072


### 解0-3 さらに高速化
- それでも遅い気がしたので、書き込み部分をリスト内包表記にしました、多少早くなった
- 実測：27sec


In [5]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# 一部リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-03','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        [file_janome.write(str(x) + '\n') for x in t.tokenize(line, stream=True)]
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-15 06:00:28.087310
end:2018-05-15 06:00:52.632747


### 解0-4 さらに高速化・・・失敗
- 読みにくいですがぜんぶリスト内包表記にしました
- 実測：88sec
  - 遅くなった何故。。。

In [6]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-04','a',encoding='utf-8') as file_janome:
    [
        [
            file_janome.write(str(x) + '\n')
            for x in t.tokenize(y, stream=True)
        ]
        for y in file_temp
    ]

print('end:' + str(datetime.datetime.now()))

start:2018-05-15 06:00:53.349604
end:2018-05-15 06:01:18.101472


### 解1 neko.txt.janome のテキスト作成
-  形態素はタブ区切り、品詞等の分類はカンマ区切りなので成型します

In [23]:
with open ('./neko.txt.janome-04','r',encoding='utf-8') as file_temp:
    text = file_temp.readlines()

lst_janome=[]
for line in text:
    lst_janome.append(
        { 'surface': line.split('\t')[0], # 表層形
          'pos'    : line.split('\t')[1].split(',')[0], # 品詞
          'pos1'   : line.split('\t')[1].split(',')[1], # 品詞細分類1
          'base'   : line.split('\t')[1].split(',')[6]   # 基本形
        })

for i in range(10):
    print(lst_janome[i])
  

{'surface': '一', 'pos': '名詞', 'pos1': '数', 'base': '一'}
{'surface': '吾輩', 'pos': '名詞', 'pos1': '代名詞', 'base': '吾輩'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': 'は'}
{'surface': '猫', 'pos': '名詞', 'pos1': '一般', 'base': '猫'}
{'surface': 'で', 'pos': '助動詞', 'pos1': '*', 'base': 'だ'}
{'surface': 'ある', 'pos': '助動詞', 'pos1': '*', 'base': 'ある'}
{'surface': '。', 'pos': '記号', 'pos1': '句点', 'base': '。'}
{'surface': '名前', 'pos': '名詞', 'pos1': '一般', 'base': '名前'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': 'は'}
{'surface': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続', 'base': 'まだ'}



## ⚾ 31. 動詞
> 動詞の表層形をすべて抽出せよ．

## 解1 普通にループ

In [22]:
lst_res = []
for janome in lst_janome:
    if janome.get('pos') == '動詞':
        lst_res.append(janome.get('surface'))

print(lst_res[::10])    

['生れ', '煮', '見', 'し', '坐っ', '助から', '付い', 'られ', '見', '渡っ', '廻り', 'もぐり込ん', '忍び込ん', '考える', 'ねぶっ', 'れ', '云う', '来', '眺め', 'し', 'いう', '帯び', '垂らす', '出来', 'られ', '得る', 'やむを得ん', '割り込む', 'くる', '至っ', '怒っ', '来たそ', '隣り', 'いる', 'すまし', '待つ', 'し', 'おら', '吹き出す', 'やめ', 'やっ', '感ずる', '飛ぶ', 'ある', '来', '見', '執っ', 'する', 'れ', '交ぜ', '思っ', 'なっ', '打ち', '知ら', 'くれ', 'する', '養う', '心付か', '抛', 'ある', 'いる', 'し', 'せる', 'する', 'いる', '生じ', '見える', '見違える', 'なる', '吐く', 'とっ', 'とろ', 'し', 'し', '引き受ける', '這い', 'え', 'がっ', 'する', 'し', 'とっ', '儲け', 'なっ', 'い', '見え', '云う', 'する', 'せ', '得る', 'いる', 'なっ', 'ある', '力め', '出さ', '信じよ', 'いる', 'なる', 'おっ', '知ら', 'い', '笑っ', 'いる', 'いる', 'ある', 'なっ', 'こぼし', '立て', '帰る', '取ら', 'いる', '見', '持っ', '思っ', '思い', '見', '云う', '得る', '出来', '這入っ', 'かかわら', 'いう', 'し', 'わかる', 'し', 'ある', 'なっ', 'あれ', 'れ', '極め', 'なれ', 'いる', '求め', 'いる', '引張る', '食い', 'なら', 'やり', '思い', '見る', '分ら', 'せ', 'もし', '出る', '透かし', '換え', '思う', 'し', 'いる', 'あっ', 'とっ', 'いる', 'しゃくい', 'いる', 'い', '食い', '濁っ', 'ある', '利く', 'し', '上っ', '突き付け', '行っ', 'い', 'いる', '剃っ', '

## 解2. 内容表記と重複排除
- このくらいのループであればリスト内包表記のほうがいいかも
- `set()` による重複排除

In [20]:
lst_res = list(set([x.get('surface') for x in lst_janome if x.get('pos') == '動詞']))
print(lst_res[::10])

['別れ', '痛み入る', '使お', '乗せ', 'いやがれ', 'ねる', '叩きつける', '備われ', '催し', '吹き込む', '事足る', '飛び下りる', '分け', '通そ', '過ぎ去っ', 'かぶる', '択ば', '反り', 'からまっ', '除い', '許し', '利く', '損する', '択ぶ', '開き直っ', '開け', '敬い', '生ん', '傭う', '響く', '浴びせ', 'もうし', '果せる', 'つけ加え', '具え', 'ぬくもっ', '耽る', 'かぶっ', '据わっ', '伝える', '引い', '目立つ', 'はいら', '防ぐ', '列ね', 'すまし', '願い', '打ち消す', 'ござい', '志す', '寝かし', 'ぬかり', '仰向き', '遣', '控える', 'とめ', '染め出し', '飲み下し', 'あるい', 'ねじ上げ', '乗じ', 'きら', '尽くさ', '引越す', '沁む', 'いただい', 'まつわっ', '食える', '選ん', '阿', '契っ', '拡げ', '光る', '構い', '盗ら', 'あらわし', '余る', '踏み', '違い', '突き戻す', '聞かさ', '考え出す', '吐き出し', '置け', '弾く', 'たのま', 'もとめ', '浴び', '召し上がら', '来', '縫っ', '磨い', 'あか', '呼ん', '陥れる', '話そ', '写っ', 'なっ', '出来れ', '誤', '示す', '笑い', 'ふり', '敲き', '祝う', '済し', '無くなっ', '浴びろ', 'かすん', '滑り込む', '殖え', '食わす', '立ちすくん', '痛み入っ', '下し', '付けよ', '拾い', '衒い', '焚け', 'あいかわら', 'すい', '這入', 'フケ', '通す', 'あび', '語り', '持っ', 'ゆるん', '習い', '立て通す', '洩らさ', 'つけ込ん', '鳴く', 'る', 'あり', '遣っ', 'こせつい', 'もぐり込む', 'かなう', '汚す', 'こじ開ける', 'られん', '並べ立て', '至ら', 'ち', '優る', '乗ん', '釣っ', '引っ掻い', '誓っ


## ⚾ 32. 動詞の原形
> 動詞の原形をすべて抽出せよ．


## 解1. 内包表記
- 原型=基本形（base）のようです

In [21]:
st_res = list(set([x.get('base') for x in lst_janome if x.get('pos') == '動詞']))
print(lst_res[::10])

['別れ', '惚れ', '痛み入る', '分ら', '使お', '泊っ', '乗せ', 'でる', 'いやがれ', 'ひっくり返っ', 'ねる', 'たとえ', '叩きつける', '中っ', '備われ', '語る', '催し', '眺める', '吹き込む', '添う', '事足る', 'へい', '飛び下りる', '用い', '分け', '眠る', '通そ', '咎め', '過ぎ去っ', 'だし', 'かぶる', '取りあげ', '択ば', '喰う', '反り', 'なわ', 'からまっ', 'たたる', '除い', '湧い', '許し', 'かよう', '利く', '過ごす', '損する', '生れ', '択ぶ', '面し', '開き直っ', 'ゆく', '開け', 'ふくらし', '敬い', '上げよ', '生ん', '思い出す', '傭う', '就く', '響く', 'できる', '浴びせ', '枯れ', 'もうし', '冠せ', '果せる', '気が付か', 'つけ加え', 'つかれ', '具え', '思い出し', 'ぬくもっ', '曲がっ', '耽る', '返し', 'かぶっ', '臭え', '据わっ', '知れる', '伝える', '引き返す', '引い', '這入ろ', '目立つ', 'わから', 'はいら', '蘇', '防ぐ', '追いかけ', '列ね', 'きい', 'すまし', 'やむ', '願い', '住ん', '打ち消す', '取り落し', 'ござい', 'なげ', '志す', '分かれ', '寝かし', 'とぼけ', 'ぬかり', '見つかる', '仰向き', '潜り込ん', '遣', '敲い', '控える', '刈り込ま', 'とめ', 'よる', '染め出し', 'のしかかっ', '飲み下し', '懸ら', 'あるい', '動か', 'ねじ上げ', '考え付い', '乗じ', 'ぶんなぐる', 'きら', '違っ', '尽くさ', 'ふっ', '引越す', 'くれれ', '沁む', 'とち', 'いただい', '戴い', 'まつわっ', '残し', '食える', '解せ', '選ん', '吐き', '阿', 'はずれ', '契っ', '倍し', '拡げ', '騒が', '光る', '脱ぎ', '構い', '掻い込ん', '盗ら', 

## ⚾ 33. サ変名詞
> サ変接続の名詞をすべて抽出せよ．

## 解1. 内包表記
- 品詞細分類1がサ変接続のものを探します

In [24]:
lst_res = list(set([x.get('base') for x in lst_janome if x.get('pos1') == 'サ変接続']))
print(lst_res[::10])

['学問', '挽回', '予言', '一段落', '退屈', '入籍', '履行', '腐爛', '持続', '養成', '徹', '転用', '推知', '行為', '束縛', '彩色', '失策', '退却', '畏服', '勘定', '差別', '喜悦', '登山', '観察', '奮闘', '顧慮', '粘着', '応対', '判断', '嘆息', '感応', '冷笑', '準備', '虚構', '手分け', '拘泥', '平身低頭', '在宿', '頂戴', '訓練', '餓死', '治療', '自認', '訪問', '発射', '立腹', '維持', '制限', '勘弁', '貯蓄', '横行', '成仏', '仮定', '賛同', '熟視', '探偵', '侵蝕', '増減', '速断', '塗抹', '顛倒', '筆誅', '当番', '治', '存在', '朗吟', '衝突', '持', '処刑', 'いらいら', '欠勤', '膠着', '放擲', '構成', '包含', '帰着', '寄附', '帰宅', '微笑', '弄', '下宿', '一貫', '生息', '厚遇', '発展', '巡回', '関', '懺悔', 'ストライキ', '叙述', '再考', '忠告', '着付け', '頓服', '識別', '激', '納得', '感銘', '突起', '説教', '窮', '全滅', '恋慕', '痙攣', '往来', '命名', '供', '自慢', '居眠り', '征伐', '閉塞', '拱手', '白黒', '水泳', '志望', '入浴', '匹敵', '生長', '誘惑', '披露', '参上', '真似', '——、', '興行', '区別', '帰臥', '朝食', '征']


## ⚾ 34. 「AのB」
> 2つの名詞が「の」で連結されている名詞句を抽出せよ．

## 解1. 普通にループ
- 名詞 の 名詞 を探します

In [25]:
lst_res = []
for i,janome in enumerate(lst_janome):
    if janome.get('base') == 'の':
        if (lst_janome[i-1].get('pos') == '名詞') and (lst_janome[i+1].get('pos') == '名詞'):
          lst_res.append(lst_janome[i-1].get('base') + janome.get('base') + lst_janome[i+1].get('base'))

print(lst_res[::20])

['彼の掌', '彼の書生', '主人の傍', '家の書生', '今更のよう', '主人の彩色', '純粋の黒', '吾輩の心臓', '彼の鼻', '黒の子分', '元の通り', 'ハリソンの歴史', '赤松の間', '主人のよう', '吾輩の主人', '門の格子', '実のところ', '後の猫', '壺の中', '障子の隙', '主人のよう', '主人の心', '二絃琴の師匠', '贅沢の結果', '二の真理', '在来の通り', '欣羨の意', '理詰の虚言', '初春の長閑', '主人の笑い声', 'つもりのところ', '方面の研究', '娼家の下婢', '念の体', '男の事', '感謝の意', '世紀の今日', '自分の境遇', '毛の病気', '平等の水彩', 'トチメンボーの復讐', '明治の文壇', '他の松', '博士の夫人', '遠くの方', '門の内', '水の労', '縮緬の羽織', '主人の我儘', '障子のうち', '教師の所', '吾輩の写真', '居士の事', '自分の家', '先刻の不平', '冊の内', '一流の論理', '迷亭の専断', '縄の一端', '御存じの平均', '埒の方', '女の声', '金田の倉', '私の家', '寒月の方', '車屋の神', '地球の磁気', '枚の絵葉書', '月並の標本', 'ステッキの代り', '活眼の士', '尻尾の先', '無用の長物', '車夫の声', '夜光の明', 'さんの話', '長吉の方', '尻尾の先', '忠告の通り', '垣の隙', '護の恐れ', '眉の根', '鼻の持主', '何のため', '空の縄張', '雪隠の横', '色の口', '僧の癖', '奴の娘', '天気の日', '右手の指', '供の袖', '時の印象', '鼻の中', '自分のため', '心の故', '下の方', '腹の内', '人の悪口', '本人の随意', '迷亭の食い気', '元のよう', '迷亭の答え', '近来の珍', 'ものの息女', '派の元祖', '智識の問屋', '贔負の尊公', '雪の夜', '目下の状態', '鼠の糞', '狼藉の練', '柳行李の間', '俗人の考', '失敗の痕', '神の製作', '未来の事件', '柳行李の辺', '山の芋の箱', '寝室の方', '

## ⚾ 35. 名詞の連接
> 名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

## 解1. 普通にループ
- 名詞 の 名詞 を探します

In [26]:
lst_res = []
for i,janome in enumerate(lst_janome):
    if janome.get('base') == 'の':
        if (lst_janome[i-1].get('pos') == '名詞') and (lst_janome[i+1].get('pos') == '名詞'):
          lst_res.append(lst_janome[i-1].get('base') + janome.get('base') + lst_janome[i+1].get('base'))

print(lst_res[::20])

['彼の掌', '彼の書生', '主人の傍', '家の書生', '今更のよう', '主人の彩色', '純粋の黒', '吾輩の心臓', '彼の鼻', '黒の子分', '元の通り', 'ハリソンの歴史', '赤松の間', '主人のよう', '吾輩の主人', '門の格子', '実のところ', '後の猫', '壺の中', '障子の隙', '主人のよう', '主人の心', '二絃琴の師匠', '贅沢の結果', '二の真理', '在来の通り', '欣羨の意', '理詰の虚言', '初春の長閑', '主人の笑い声', 'つもりのところ', '方面の研究', '娼家の下婢', '念の体', '男の事', '感謝の意', '世紀の今日', '自分の境遇', '毛の病気', '平等の水彩', 'トチメンボーの復讐', '明治の文壇', '他の松', '博士の夫人', '遠くの方', '門の内', '水の労', '縮緬の羽織', '主人の我儘', '障子のうち', '教師の所', '吾輩の写真', '居士の事', '自分の家', '先刻の不平', '冊の内', '一流の論理', '迷亭の専断', '縄の一端', '御存じの平均', '埒の方', '女の声', '金田の倉', '私の家', '寒月の方', '車屋の神', '地球の磁気', '枚の絵葉書', '月並の標本', 'ステッキの代り', '活眼の士', '尻尾の先', '無用の長物', '車夫の声', '夜光の明', 'さんの話', '長吉の方', '尻尾の先', '忠告の通り', '垣の隙', '護の恐れ', '眉の根', '鼻の持主', '何のため', '空の縄張', '雪隠の横', '色の口', '僧の癖', '奴の娘', '天気の日', '右手の指', '供の袖', '時の印象', '鼻の中', '自分のため', '心の故', '下の方', '腹の内', '人の悪口', '本人の随意', '迷亭の食い気', '元のよう', '迷亭の答え', '近来の珍', 'ものの息女', '派の元祖', '智識の問屋', '贔負の尊公', '雪の夜', '目下の状態', '鼠の糞', '狼藉の練', '柳行李の間', '俗人の考', '失敗の痕', '神の製作', '未来の事件', '柳行李の辺', '山の芋の箱', '寝室の方', '

TBD


## ⚾ 36. 単語の出現頻度
> 文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

## ⚾ 37. 頻度上位10語
> 出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## ⚾ 38. ヒストグラム
> 単語の出現頻度のヒストグラム（横軸に出現頻度，縦軸に出現頻度をとる単語の種類数を棒グラフで表したもの）を描け．

## ⚾ 39. Zipfの法則
> 単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．
